In [ ]:
%matplotlib inline

UsageError: Line magic function `%` not found.


#### 数据并行

在这个教程里，我们将学习如何使用`DataParallel`来使用多GPU。

PyTorch非常容易就可以使用多GPU，用如下方式把一个模型放到GPU上：

In [ ]:
deivce = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

然后复制所有的张量到GPU上：

In [2]:
mytensor = my_tensor.to(device)

NameError: name 'my_tensor' is not defined

请注意，只调用`my_tensor.to(device)`并没有复制张量到GPU上，而是返回了一个copy。所以需要把它赋值给一个新的张量并在GPU上使用这个张量。

在多GPU上执行前向和反向传播是自然而然的事。但是PyTorch默认将只使用一个GPU。

使用`DataParallel`可以轻易地让模型并行运行在多个GPU上。

In [ ]:
model = nn.DataParallel(model)

#### 导入和参数

导入PyTorch模块和定义参数。

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 参数
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device

#### 虚拟数据集

制作一个虚拟（随机）数据集，你只需要实现`__getitem__`

In [ ]:
class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return self.len

rand_loader = DataLoader(dataset = RandomDataset(input_size, data_size),
                         batch_size = batch_size, shuffle = True)

#### 简单模型

模型只接受一个输入，执行一个线性操作，然后得到结果。说明：`DataParallel`能在任何模型（CNN， RNN， Capsule Net等）上使用。

我们在模型内部放置了一条打印语句来打印输入和输出向量的大小。

In [ ]:
class Model(nn.Module):

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)
        print("Input Size :", input.size(),
              "Output Size :", output.size())
        
        return output

NameError: name 'nn' is not defined

#### 创建一个模型和数据并行

首先，我们需要创建一个模型实例和检测我们是否有多个GPU。如果有多个GPU，使用`nn.DataParallel`来包装我们的模型。然后通过`model.to(device)`把模型放到GPU上。

In [ ]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

model.to(device)

#### 运行模型
现在可以看到输入和输出张量的大小。

In [ ]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)

    print("Outside: Input Size :", input.size(),
          "Output Size :", output.size())

#### 总结

DataParallel会自动划分数据，并将作业发送到多个GPU上的多个模型。并在每个模型完成作业后，收集合并结果并返回。